<a href="https://colab.research.google.com/github/naripok/tpu_transformer_colab/blob/master/tpu_transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/tensorflow/examples/blob/master/community/en/transformer_chatbot.ipynb#scrollTo=rHMPkA2eQrpT

In [1]:
#@title Transformer Network Chatbot Training Routine {display-mode: "form"}


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#@title Environment Setup {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import output

import re
import logging
import pickle
from tqdm import tqdm
import tensorflow as tf
from tensorflow_datasets.core.features.text import SubwordTextEncoder
tf.random.set_seed(1234)
logging.basicConfig(level=logging.INFO)

tf.keras.backend.clear_session()
with output.use_tags('setup'):

    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running ovariable_namen TPU ', tpu.cluster_spec().as_dict()['worker'])
    except ValueError:
        raise BaseException('ERROR: Not connected to a TPU runtime')

    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

output.clear(output_tags='setup')
logging.info('Done!')

INFO:root:Done!


In [0]:
#@title Training Parameters {display-mode: "form"}

# This code will be hidden when the notebook is loaded.


NEW_MODEL = True  #@param {type:"boolean"}
TRAIN = True  #@param {type:"boolean"}

# Training params
EPOCHS = 500  #@param
BATCH_SIZE = 128 * tpu_strategy.num_replicas_in_sync
BUFFER_SIZE = 10000  #@param
WARMUP_STEPS = 10000  #@param
MIN_DELTA = 0.001  #@param
PATIENCE = 30  #@param
BASELINE = None  #@param

# tokenizer params
TARGET_VOCAB_SIZE = 2**13  #@param

# Maximum number of samples to preprocess
MAX_LENGTH = 32  #@param
MAX_SAMPLES =   300000#@param

# Hyper-parameters
NUM_LAYERS =   2#@param
D_MODEL = 256  #@param
NUM_HEADS =   8#@param
UNITS = 512  #@param
DROPOUT = 0.1  #@param

In [0]:
#@title Save path {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

import os
import datetime

model_path = "/content/drive/My Drive/discordbot/saved_model"  #@param {type:"string"}
model_weights_path = model_path + '/weights.h5'
tokenizer_path = model_path + '/saved_tokenizer.pickle'
log_dir = model_path + '/logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [0]:
#@title Preprocessing functions {display-mode: "form"}

# This code will be hidden when the notebook is loaded.


def tokenize_and_filter(tokenizer, inputs, outputs, max_length=32):
    # Tokenize, filter and pad sentences

    # Define start and end token to indicate the start and end of a sentence
    START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

    logging.info('Tokenizing sentences.')

    tokenized_inputs, tokenized_outputs = [], []
    for (sentence1, sentence2) in tqdm(zip(inputs, outputs)):
        # tokenize sentence
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

        # check tokenized sentence max length
        if len(sentence1) <= max_length and len(sentence2) <= max_length:
          tokenized_inputs.append(sentence1)
          tokenized_outputs.append(sentence2)

    # pad tokenized sentences
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=max_length, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=max_length, padding='post')

    logging.info('Done!')

    return tokenized_inputs, tokenized_outputs


def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    sentence = re.sub(r'([?.!,])', r' \1 ', sentence)
    sentence = re.sub(r'[" "]+', ' ', sentence)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r'[^a-zA-Z?.!,]+', ' ', sentence)
    sentence = sentence.strip()
    # adding a start and an end token to the sentence
    return sentence


def load_conversations(max_samples, eval_percent=0.1):
    logging.info('Loading data.')

    def split_data(inputs, outputs, eval_percent):
        eval_index = int(len(inputs) * (1 - eval_percent))
        return (inputs[:eval_index], outputs[:eval_index], inputs[eval_index:], outputs[eval_index:])

    # Download training data
    path_to_zip = tf.keras.utils.get_file(
        'cornell_movie_dialogs.zip',
        origin='http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip',
        extract=True)

    path_to_dataset = os.path.join(
        os.path.dirname(path_to_zip), "cornell movie-dialogs corpus")

    path_to_movie_lines = os.path.join(path_to_dataset, 'movie_lines.txt')
    path_to_movie_conversations = os.path.join(path_to_dataset, 'movie_conversations.txt')

    # dictionary of line id to text
    id2line = {}
    with open(path_to_movie_lines, errors='ignore') as file:
        lines = file.readlines()

    logging.info('Done!')
    logging.info('Preprocessing data.')

    for line in lines:
        parts = line.replace('\n', '').split(' +++$+++ ')
        id2line[parts[0]] = parts[4]

    inputs, outputs = [], []
    with open(path_to_movie_conversations, 'r') as file:
        lines = file.readlines()

    for line in tqdm(lines):
        parts = line.replace('\n', '').split(' +++$+++ ')
        # get conversation in a list of line ID
        conversation = [line[1:-1] for line in parts[3][1:-1].split(', ')]

        for i in range(len(conversation) - 1):
            inputs.append(preprocess_sentence(id2line[conversation[i]]))
            outputs.append(preprocess_sentence(id2line[conversation[i + 1]]))

            if len(inputs) >= max_samples:
                logging.info('Done!')
                return split_data(inputs, outputs, eval_percent)

    logging.info('Done!')
    return split_data(inputs, outputs, eval_percent)

In [0]:
#@title Model Definition {display-mode: "form"}

# This code will be hidden when the notebook is loaded.


def scaled_dot_product_attention(query, key, value, mask):
    ''' Calculate the attention weights. '''

    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name='multi_head_attention'):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], \
        inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs


def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]


def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)


class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        # apply sin to even index in the array
        sines = tf.math.sin(angle_rads[:, 0::2])
        # apply cos to odd index in the array
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


def encoder_layer(units, d_model, num_heads, dropout, name='encoder_layer'):
    inputs = tf.keras.Input(shape=(None, d_model), name='inputs')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention = MultiHeadAttention(
        d_model, num_heads, name='attention')({
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': padding_mask
        })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)


def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='encoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='encoder_layer_{}'.format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)


def decoder_layer(units, d_model, num_heads, dropout, name='decoder_layer'):
    inputs = tf.keras.Input(shape=(None, d_model), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
        d_model, num_heads, name='attention_1')(inputs={
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': look_ahead_mask
        })
    attention1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
        d_model, num_heads, name='attention_2')(inputs={
            'query': attention1,
            'key': enc_outputs,
            'value': enc_outputs,
            'mask': padding_mask
        })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)


def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):

    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)


def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name='transformer'):

    inputs = tf.keras.Input(shape=(None,), name='inputs')
    dec_inputs = tf.keras.Input(shape=(None,), name='dec_inputs')

    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='enc_padding_mask')(inputs)

    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)

    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size, name='outputs')(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)


def loss_function(max_length=32):
    def _loss_function(y_true, y_pred):
        y_true = tf.reshape(y_true, shape=(-1, max_length - 1))

        loss = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction='none')(y_true, y_pred)

        mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
        loss = tf.multiply(loss, mask)

        return tf.reduce_mean(loss)
    return _loss_function


class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.d_model_t = tf.cast(self.d_model, tf.float32)

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model_t) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {
            'd_model': self.d_model,
            'warmup_steps': self.warmup_steps
        }


def accuracy(max_length=32):
    def _accuracy(y_true, y_pred):
        # ensure labels have shape (batch_size, max_length - 1)
        y_true = tf.reshape(y_true, shape=(-1, max_length - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)
    return _accuracy


def make_tokenizer(data, target_vocab_size=2**13):
    logging.info('Training tokenizer...')

    tokenizer = SubwordTextEncoder.build_from_corpus(data,
            target_vocab_size=target_vocab_size)

    logging.info(f'Target Tokenizer vocab size: {target_vocab_size}')
    logging.info(f'Tokenizer vocab size: {tokenizer.vocab_size}')

    # save tokenizer
    logging.info('Saving tokenizer.')

    if not os.path.exists(model_path):
        os.makedirs(model_path)

    with open(tokenizer_path, 'wb+') as f:
        pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

    # tokenizer.save_to_file(tokenizer_path)

    logging.info('Done!')

    return tokenizer


def make_dataset(
    inputs,
    outputs,
    tokenizer=None,
    batch_size=128,
    buffer_size=20000,
    max_length=32,
    target_vocab_size=2**13):

    if not tokenizer:
        tokenizer = make_tokenizer(inputs + outputs, target_vocab_size)

    inputs, outputs = tokenize_and_filter(tokenizer, inputs, outputs, max_length)

    logging.info('Making data set.')
    # decoder inputs use the previous target as input
    # remove START_TOKEN from targets
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'inputs': inputs,
            'dec_inputs': outputs[:, :-1]
        },
        {
            'outputs': outputs[:, 1:]
        },
    ))

    dataset = dataset.cache() \
                    .shuffle(buffer_size) \
                    .batch(batch_size) \
                    .prefetch(tf.data.experimental.AUTOTUNE)

    return tokenizer, dataset


def make_model(
    tokenizer=None, 
    num_layers=2,
    units=512,
    d_model=256,
    num_heads=8,
    dropout=0.1,
    max_length=32,
    warmup_steps=4000):

    
    logging.info('Compiling model.')
    learning_rate = CustomSchedule(d_model, warmup_steps)

    optimizer = tf.keras.optimizers.Adam(
        learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    # Vocabulary size plus start and end token
    vocab_size = tokenizer.vocab_size + 2

    with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
        model = transformer(
            vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout
            )

        model.compile(optimizer=optimizer, loss=loss_function(max_length),
                      metrics=[accuracy(max_length)])

    return model


def train(model, train_data, eval_data, epochs=10, min_delta=0.001,
          patience=10, baseline=None):
    
    # reset session
    # tf.keras.backend.clear_session()

    # training callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=min_delta, patience=patience,
        verbose=1, mode='auto', baseline=baseline,
        restore_best_weights=True
        )
    tensor_board = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, histogram_freq=0, write_graph=False, write_images=True,
        update_freq=50, profile_batch=2, embeddings_freq=10,
        embeddings_metadata=None
        )
    terminate_on_nan = tf.keras.callbacks.TerminateOnNaN()

    # Create a callback that saves the model's weights
    logging.info('Training model.')
    try:
        model.fit(
                train_data,
                validation_data=eval_data,
                epochs=epochs,
                callbacks=[
                    early_stopping,
                    # tensor_board,
                    terminate_on_nan
                    ]
                )

    except KeyboardInterrupt:
        logging.info('\nTraining Interruped!')

    finally:
        logging.info('Saving model.')
        model.save_weights(model_weights_path, overwrite=True)

    return model


def evaluate(tokenizer, model, sentence, max_length, training=False):
    start_token, end_token = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

    sentence = preprocess_sentence(sentence)
    sentence = tf.expand_dims(start_token + tokenizer.encode(sentence) + end_token, axis=0)
    output = tf.expand_dims(start_token, 0)

    for i in range(max_length):
        predictions = model(inputs=[sentence, output], training=training)

        # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # return the result if the predicted_id is equal to the end token
        if tf.equal(predicted_id, end_token[0]):
            break

        # concatenated the predicted_id to the output which is given to the decoder
        # as imodel_optsts input.
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0)


def predict(tokenizer, model, sentence, max_length, training=False):
    prediction = evaluate(tokenizer, model, sentence, max_length, training)
    prediction = [i for i in prediction if i < tokenizer.vocab_size]
    return tokenizer.decode(prediction)


def load_model(model_opts):
    logging.info('Loading tokenizer.')
    with open(tokenizer_path, 'rb') as f:
        tokenizer = pickle.load(f)

    # tokenizer = SubwordTextEncoder.load_from_file(tokenizer_path)

    logging.info('Loading model.')
    model = make_model(tokenizer, **model_opts)
    model.load_weights(model_weights_path)

    logging.info('Done!')
    return tokenizer, model

In [0]:
#@title Training {display-mode: "form"}

# This code will be hidden when the notebook is loaded.


train_opts = {
    'epochs': EPOCHS,
    'patience': PATIENCE,
    'min_delta': MIN_DELTA,
    'baseline': BASELINE
}

dataset_opts = {
    'batch_size': BATCH_SIZE,
    'buffer_size': BUFFER_SIZE,
    'max_length': MAX_LENGTH,
    'target_vocab_size': TARGET_VOCAB_SIZE
}

model_opts = {
    'num_layers': NUM_LAYERS,
    'units': UNITS,
    'd_model': D_MODEL,
    'num_heads': NUM_HEADS,
    'dropout': DROPOUT,
    'max_length': MAX_LENGTH,
    'warmup_steps': WARMUP_STEPS,
}

train_questions, train_answers, eval_questions, eval_answers = load_conversations(MAX_SAMPLES)

if NEW_MODEL:
    tokenizer, train_data = make_dataset(train_questions, train_answers,
                                         max_length=MAX_LENGTH, target_vocab_size=TARGET_VOCAB_SIZE)
    _, eval_data = make_dataset(eval_questions, eval_answers, tokenizer,
                                max_length=MAX_LENGTH, target_vocab_size=TARGET_VOCAB_SIZE)
    
    model = make_model(tokenizer, **model_opts)
    model.summary()

    if TRAIN:
        model = train(model, train_data, eval_data, **train_opts)
else:
    tokenizer, model = load_model(model_opts)
    model.summary()

    if TRAIN:
        tokenizer, train_data = make_dataset(train_questions, train_answers,
                                             max_length=MAX_LENGTH, target_vocab_size=TARGET_VOCAB_SIZE)
        _, eval_data = make_dataset(eval_questions, eval_answers, tokenizer,
                                    max_length=MAX_LENGTH, target_vocab_size=TARGET_VOCAB_SIZE)
        
        model = train(model, train_data, eval_data, **train_opts)

INFO:root:Loading data.
INFO:root:Done!
INFO:root:Preprocessing data.
100%|██████████| 83097/83097 [00:07<00:00, 11027.84it/s]
INFO:root:Done!
INFO:root:Training tokenizer...
INFO:absl:SubwordTextEncoder build: trying min_token_count 113574
INFO:absl:SubwordTextEncoder build: trying min_token_count 56787
INFO:absl:SubwordTextEncoder build: trying min_token_count 28393
INFO:absl:SubwordTextEncoder build: trying min_token_count 14196
INFO:absl:SubwordTextEncoder build: trying min_token_count 7098
INFO:absl:SubwordTextEncoder build: trying min_token_count 3549
INFO:absl:SubwordTextEncoder build: trying min_token_count 1774
INFO:absl:SubwordTextEncoder build: trying min_token_count 887
INFO:absl:SubwordTextEncoder build: trying min_token_count 443
INFO:absl:SubwordTextEncoder build: trying min_token_count 221
INFO:absl:SubwordTextEncoder build: trying min_token_count 110
INFO:absl:SubwordTextEncoder build: trying min_token_count 55
INFO:absl:SubwordTextEncoder build: trying min_token_count

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Model)                 (None, None, 256)    3139328     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [0]:
#@title Talk to the model { vertical-output: true }
you = "maybe you need some more training?" #@param {type:"string"}
print(f'transformer: {predict(tokenizer, model, you, max_length=MAX_LENGTH)}')